In [1]:
import numpy as np
import pandas as pd

In [2]:
reviews_data = pd.read_excel('./data/Third Data.xlsx', sheet_name='Original Sheet (no breakdown)')
reviews_data = reviews_data.iloc[1:, :].drop(columns=['Column1']).rename(columns={'Column2': 'URL', 'Column3': 'review_text'}).fillna('').reset_index(drop=True)

keywords_data = pd.read_excel('./data/hugging.xlsx', sheet_name='Sheet1')

In [3]:
from review_text import ReviewText, Sentence
from label import get_labels
from utils import flatten
labels_data, breakdowns_regardless = get_labels()


2022-06-21 14:01:43.651932: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-06-21 14:01:43.651967: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [5]:
class CategorizeReviewTextLabel:
  def __init__(self, review_text, label):
    # create a Review Text instance
    self.review_text = ReviewText(review_text)

    self.label = label
    self.label_type = self.detect_label_type()
    if self.label_type != 0:
      self.label_data = labels_data[label]['data']
    self._keywords = []
    self.FOUND_KEYWORD = False

  @property
  def keywords(self):
    # flatten the keywords & remove duplicate keyword (use set) before return
    return list(set(flatten(self._keywords)))

  def get_label_data(self, key, flat=False):
    data = self.label_data[key]
    if flat:
      data = flatten(data)
      data = [value for value in data if value != '']
    return data

  def detect_label_type(self):
    return labels_data[self.label]['type']

  def find_keywords_no_sentiment(self, keywords, change_to_keywords):
    results = []
    for keywords_index, keywords_present in enumerate(keywords):
        for keyword in keywords_present:
          if keyword.lower() in self.review_text.text.lower():
            corresponding_keyword = change_to_keywords[keywords_index]
            results.append(corresponding_keyword)
    return results

  def add_otherwise_keyword(self):
    if 'OTHERWISE' in self.label_data:
      # get the keyword
      if len(self.label_data['OTHERWISE']) < 2:
        otherwise_keyword = self.label_data['OTHERWISE'][0]
      else:
        otherwise_keyword = self.label_data['OTHERWISE'][self.review_text.sentiment]
      self._keywords.append(otherwise_keyword)

  def processing(self):
    # label_type = 0 => not run
    if self.label_type == 0:
      return

    if self.label_type == 1:
      self.processing_type_1()
    elif self.label_type == 2:
      self.processing_type_2()
    elif self.label_type == 3:
      self.processing_type_3()
    else:
      self.processing_type_4()
    self.processing_regardless()

    # if there are no keyword in the review text, add OTHERWISE keywords
    if not self.FOUND_KEYWORD:
      self.add_otherwise_keyword()

  def processing_type_1(self):
    FIND_KEYWORDS = self.get_label_data('FIND KEYWORDS')
    CORRESPONDING_KEYWORDS = self.get_label_data('RUN SENTIMENT ANALYSIS')
    OPPOSITE_WORDS = None

    for find_keywords_index, find_keywords in enumerate(FIND_KEYWORDS):
      for find_keyword in find_keywords:
        if find_keyword.lower() in self.review_text.text.lower():
          # find correspoding keyword
          corresponding_keyword = CORRESPONDING_KEYWORDS[find_keywords_index]
          # loop through all sentences and find a sentence has that keyword
          for sentence in self.review_text.sentences:
            if find_keyword.lower() in sentence.text.lower():
              self._keywords.append(corresponding_keyword[sentence.sentiment])
          self.FOUND_KEYWORD = True

    # EXCEPTION: if label is consistency and texture
    if self.label == 'consistency and texture':
      NO_SENTIMENT_KEYWORDS = self.get_label_data('IF KEYWORDS PRESENT NO SENTIMENT', flat=True)
      for no_sentiment_keyword in NO_SENTIMENT_KEYWORDS:
        if no_sentiment_keyword in self.review_text.text:
          self._keywords.append(no_sentiment_keyword)
          self.FOUND_KEYWORD = True
    

    # handle the exception
    if 'BREAKDOWN REGARDLESS OF SENTIMENT' in self.label_data:
      for keyword_breakdown_regardless in self.get_label_data('BREAKDOWN REGARDLESS OF SENTIMENT'):
        if keyword_breakdown_regardless in self.review_text.text:
          self._keywords.append(keyword_breakdown_regardless)
    
    if 'IF WORDS PRESENT' in self.label_data:
      self._keywords.append(self.find_keywords_no_sentiment(self.get_label_data('IF WORDS PRESENT'),
                                                      self.get_label_data('CHANGE TO')))
      

  def processing_type_2(self):
    FIND_KEYWORDS = self.label_data['IF KEYWORDS PRESENT NO SENTIMENT']
    CORRESPONDING_KEYWORDS = self.label_data['CHANGE TO']
    results = self.find_keywords_no_sentiment(FIND_KEYWORDS, CORRESPONDING_KEYWORDS)
    if len(results) > 0:
      self.FOUND_KEYWORD = True
    self._keywords.append(results)

    # EXCEPTION: nightly usage
    if self.label == 'nightly usage':
      if 'overnight' in self.keywords:
        if 'night' in self.keywords:
          self._keywords = [keyword for keyword in self.keywords if keyword != 'night']

  def processing_type_3(self):
    if 'IF KEYWORDS PRESENT' not in self.label_data:
      FIND_KEYWORDS = self.label_data['RUN SENTIMENT ANALYSIS ON WHOLE REVIEW']
      keyword = FIND_KEYWORDS[self.review_text.sentiment]
      self._keywords.append(keyword)
      self.FOUND_KEYWORD = True

    else:
      TAKE_FIRST = None
      KEYWORD_FOUND = {}

      # EXCEPTION: time of usage
      if self.label == 'time of usage':
        TAKE_FIRST = True

      # handle exception
      FIND_KEYWORDS = self.label_data['IF KEYWORDS PRESENT']
      CORRESPONDING_KEYWORDS = self.label_data['ASSIGN SENTIMENT']
      for find_keywords_index, find_keywords in enumerate(FIND_KEYWORDS):
        for find_keyword in find_keywords:
          if find_keyword.lower() in self.review_text.text.lower():
            # find correspoding keyword
            if len(CORRESPONDING_KEYWORDS[find_keywords_index]) > 1:
              corresponding_keyword = CORRESPONDING_KEYWORDS[find_keywords_index][self.review_text.sentiment]
            else:
              corresponding_keyword = CORRESPONDING_KEYWORDS[find_keywords_index][0]

            # if EXCEPTION: time of usage, try to find first keyword
            if TAKE_FIRST:
              position_found = self.review_text.text.lower().index(find_keyword.lower())
              
              if KEYWORD_FOUND == {}:
                KEYWORD_FOUND = {'keyword': corresponding_keyword, 'position': position_found}
              else:
                if position_found < KEYWORD_FOUND['position']:
                  KEYWORD_FOUND = {'keyword': corresponding_keyword, 'position': position_found}
            else:
              self._keywords.append(corresponding_keyword)
              self.FOUND_KEYWORD = True

      if TAKE_FIRST:
        if 'keyword' in KEYWORD_FOUND:
          self._keywords.append(KEYWORD_FOUND['keyword'])
          self.FOUND_KEYWORD = True
      
      if 'IF KEYWORDS PRESENT NO SENTIMENT' in self.label_data:
        for keywords_index, keywords in enumerate(self.label_data['IF KEYWORDS PRESENT NO SENTIMENT']):
          for keyword in keywords:
            if keyword.lower() in self.review_text.text.lower():
              corresponding_keyword = self.label_data['CHANGE TO'][keywords_index]
              self._keywords.append(corresponding_keyword)
    
  def processing_type_4(self):
    CHANGE_TO = self.label_data['CHANGE TO']

    for keyword in flatten(CHANGE_TO):
      self._keywords.append(keyword)

  def processing_regardless(self):
    for idx, breakdown in enumerate(breakdowns_regardless):
      FIND_KEYWORDS = breakdown['FOR THE WORD']

      if idx == 0:
        CORRESPONDING_KEYWORDS = breakdown['RUN SENTIMENT ANALYSIS']
        OPPOSITE_WORDS = breakdown['OPPOSITE WORDS']
        
        for find_keywords_index, find_keywords in enumerate(FIND_KEYWORDS):
          for find_keyword in find_keywords:
            if find_keyword.lower() in self.review_text.text.lower():
              # find correspoding keyword
              corresponding_keyword = CORRESPONDING_KEYWORDS[find_keywords_index]

              # loop through all sentences and find a sentence has that keyword
              for sentence in self.review_text.sentences:
                if find_keyword.lower() in sentence.text.lower():
                  FIND_OPPOSITE = False
                  for opposite_word in OPPOSITE_WORDS:
                    k = '{0} {1}'.format(opposite_word, find_keyword.lower())
                    if k in sentence.text.lower():
                      self._keywords.append(corresponding_keyword[0])
                      FIND_OPPOSITE = True
                  if not FIND_OPPOSITE:
                    self._keywords.append(corresponding_keyword[sentence.sentiment])

      if idx == 1:
        CORRESPONDING_KEYWORDS = breakdown['RERUN CATEGORY']
        
        temp_label = self.label
        temp_label_data = self.label_data
        temp_found_keyword = self.FOUND_KEYWORD

        for find_keywords_index, find_keywords in enumerate(FIND_KEYWORDS):
          for find_keyword in find_keywords:
            if find_keyword.lower() in self.review_text.text.lower():
              self.label = flatten(CORRESPONDING_KEYWORDS)[find_keywords_index]
              self.label_data = labels_data[self.label]['data']
              self.FOUND_KEYWORD = False
              self.processing_type_1()
              if not self.FOUND_KEYWORD:
                self.add_otherwise_keyword()
        
        self.FOUND_KEYWORD = temp_found_keyword
        self.label = temp_label
        self.label_data = temp_label_data
      
      if idx == 2:
        CORRESPONDING_KEYWORDS = flatten(breakdown['RUN SENTIMENT ANALYSIS'])
        
        if self.label == 'cleansers and cleansing':
          continue

        for keyword in flatten(FIND_KEYWORDS):
          if keyword in self.review_text.text.lower():
            for sentence in self.review_text.sentences:
              if keyword.lower() in sentence.text.lower():
                self._keywords.append(CORRESPONDING_KEYWORDS[sentence.sentiment])
      


In [7]:
review_text = """
  I read the reviews before day light react I night bought only using hoo creamy stinging
  """
labels = ['sensitive skin (type)']
#labels = ['time of usage']

for label in labels:
  categorized_review_text = CategorizeReviewTextLabel(review_text, label)
  categorized_review_text.processing()
  print(categorized_review_text.keywords)

['no irritation', 'light', 'sensitive skin (positive)', 'creamy']


In [96]:
from tqdm import tqdm

# only keep url and review_text
results_df = reviews_data.drop(reviews_data.columns[2:], axis=1)
reviews_data = reviews_data.iloc[:5, :]

for idx, row in tqdm(reviews_data.iterrows()):
  url, review_text = row['URL'], row['review_text']
  review_labels = [label for label in row[2:] if isinstance(label, str)]

  keywords = []
  for label in review_labels:
    if label in labels_data:
      categorized_review_text = CategorizeReviewTextLabel(review_text, label)
      categorized_review_text.processing()
      keywords.append(categorized_review_text.keywords)

  for idx_keyword, keyword in enumerate(list(set(flatten(keywords)))):
    keyword_column = 'Keyword {0}'.format(idx_keyword + 1)
    if keyword_column not in results_df.columns:
      results_df[keyword_column] = ''
    results_df.loc[idx, keyword_column] = keyword

5it [00:02,  2.22it/s]


In [97]:
results_df


,URL,review_text,Keyword 1,Keyword 2,Keyword 3,Keyword 4,Keyword 5,Keyword 6,Keyword 7
0,https://www.meccabeauty.co.nz/mario-badescu/hy...,Lockdown eyes are super bright. The appearan...,reduced wrinkles,covered/reduced eye circles,shine/bright/lumi/glow/rad,,,,
1,https://www.meccabeauty.co.nz/mario-badescu/hy...,It feels so soothing around my eyes and help...,reduced wrinkles,cooling/soothing,absorb (positive),covered/reduced eye circles,,,
2,https://www.meccabeauty.co.nz/mario-badescu/hy...,I bought this a couple of weeks ago along wi...,glide makeup (positive),covered/reduced eye circles,hydrating/moisturising,shine/bright/lumi/glow/rad,rich,weeks (positive),as/ds/pig/disc (positive)
3,https://www.meccabeauty.co.nz/mario-badescu/hy...,I brought this a few weeks ago and have used...,weeks (positive),covered/reduced eye circles,glide makeup (positive),hydrating/moisturising,,,
4,https://www.meccabeauty.co.nz/mario-badescu/hy...,This product is so nice to put on after a bi...,covered/reduced eye circles,night,shine/bright/lumi/glow/rad,days (positive),calming,,
...,...,...,...,...,...,...,...,...,...
995,https://www.meccabeauty.co.nz/kiehls/midnight-...,Bought this a month ago with concerns that I...,,,,,,,
996,https://www.meccabeauty.co.nz/kiehls/midnight-...,Love it's light texture that goes a long way...,,,,,,,
997,https://www.meccabeauty.co.nz/kiehls/midnight-...,This cream makes miraculous for my eyes and ...,,,,,,,
998,https://www.meccabeauty.co.nz/kiehls/midnight-...,This is probably my first ever review but I ...,,,,,,,
